<a href="https://colab.research.google.com/github/echung2/echung2/blob/master/sentence_bert_topicmodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers sentence-transformers umap-learn scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.1 MB/s eta 0:00:00


In [2]:

# Import libraries
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import umap

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Import library for file uploading
from google.colab import files


In [4]:
# Upload the file
uploaded = files.upload()


Saving H_Miyaha_OCR.txt to H_Miyaha_OCR.txt


In [5]:
# Load the uploaded file
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

In [6]:
# Preprocess text: remove special characters and split into sentences
import re
text = re.sub(r'[^\w\s。！？]', '', text)  # Remove special characters except Japanese sentence markers
text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
sentences = re.split(r'[。！？]', text)  # Split by Japanese sentence-ending markers
sentences = [sentence.strip() for sentence in sentences if len(sentence.strip()) > 5]  # Remove empty or very short sentences

In [7]:
# Ensure there are enough sentences for clustering
if len(sentences) < 10:
    raise ValueError("The uploaded text contains too few sentences for topic modeling. Please use a longer text.")

In [8]:
# Load BERT model for embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Generate embeddings
embeddings = model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [10]:
# Dimensionality reduction using UMAP (optional, for clustering)
reducer = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine')
reduced_embeddings = reducer.fit_transform(embeddings)

In [11]:
# Define number of topics based on data size
num_topics = min(10, len(sentences))  # Ensure n_clusters <= n_samples
print(f"Number of topics adjusted to: {num_topics}")

Number of topics adjusted to: 10


In [12]:
# Apply k-means clustering to find topics
kmeans = KMeans(n_clusters=num_topics, random_state=42)
kmeans.fit(reduced_embeddings)
labels = kmeans.labels_

In [13]:
# Assign topics to sentences in a DataFrame
topic_sentences = pd.DataFrame({
    'Sentence': sentences,
    'Topic': labels
})

In [21]:
# Restructure data so that each topic has sentences listed across columns
topic_groups = topic_sentences.groupby('Topic')['Sentence'].apply(list)
max_sentences = topic_groups.apply(len).max()  # Find maximum number of sentences in a topic

In [22]:
# Prepare data for CSV output in the desired format
output_data = pd.DataFrame({'Topic': topic_groups.index})
for i in range(max_sentences):
    output_data[f'Sentence {i+1}'] = topic_groups.apply(lambda x: x[i] if i < len(x) else "")

<ipython-input-22-72a4e84b3255>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_data[f'Sentence {i+1}'] = topic_groups.apply(lambda x: x[i] if i < len(x) else "")
<ipython-input-22-72a4e84b3255>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_data[f'Sentence {i+1}'] = topic_groups.apply(lambda x: x[i] if i < len(x) else "")
<ipython-input-22-72a4e84b3255>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [24]:


# Save the structured output to a CSV file
output_file = "test_bert_topic_modeling_results.csv"
output_data.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file}")

Results saved to test_bert_topic_modeling_results.csv


In [14]:
# Extract top keywords for each topic
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
# Define custom stop words
custom_stop_words = [
   'Page', 'は', 'が', 'の', 'に', 'を', 'へ', 'と', 'で', 'や', 'も', 'から', 'まで',
    'だけ', 'より', 'しか', 'ながら', 'など', 'これ', 'それ', 'あれ', 'どれ',
    'ここ', 'そこ', 'あそこ', 'どこ', 'そして', 'しかし', 'だから', 'なぜ',
    'どう', '何', 'いつ', 'ただ', 'ようやく', 'やはり', 'さらに', 'すぐに',
    'いる', 'ある', 'する', 'なる', '言う', '思う', '分かる', '知る'
]

In [16]:
# Combine sentences by topic
topic_groups = topic_sentences.groupby('Topic')['Sentence'].apply(' '.join)

In [17]:
# Vectorize text for keyword extraction
vectorizer = CountVectorizer(max_features=20, stop_words=custom_stop_words)
keywords_per_topic = {}

for topic, text in topic_groups.items():
    if text.strip():  # Ensure text is not empty
        vectorizer.fit([text])
        keywords = vectorizer.get_feature_names_out()
        keywords_per_topic[topic] = keywords
    else:
        print(f"Warning: Topic {topic} has empty text and will be skipped.")
        keywords_per_topic[topic] = ["No keywords available"]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['page'] not in stop_words.
  warnings.warn(


In [18]:
# Display topics and their keywords
for topic, keywords in keywords_per_topic.items():
    print(f"Topic {topic}: {', '.join(keywords)}")

Topic 0: 109, page, 囚われ人が誰だったのか, 囮にしたみたいで悪かったな, 壊れろという呪詛が時を経て脳にこだましていた, 変わった, 外出着の類も新たに買わなければならなかった, 好奇心さ, 存外億劫な作業だった, 実継にとってもその隠密行動は初耳だった, 実継は思わず隠すように手で耳朶を覆って耳を澄ませた, 実継は成田から新東亜総合病院に向かうタクシー内でヒエロに提示される情報誌の記事群, 実継もまたお, 実験と言うがこれは人類の大半を掛け金にした, 寝泊まりしている桐佳が美亜羽の介助を買って出たのだ, 察などなされていないからです, 対し神冴脳工学医療医師連絡会の一部が取った戦略は醜かった, 少し間があった, 市販のインプラントは鼻腔から注入する形式が多いが脳内に直接打ち込む方が高コストで, 師連絡会への反抗とみなされないように取り繕わねばと実継はもつれそうな舌を回す
Topic 1: 119, page, 少し俯く, 少女の後ろ姿が小さくなっていく, 少女は眉をしかめることもなく, 少女美亜羽は無言で頷く, 屋まで走ってノックもせずドアを開いた, 弱く見える今の, 彼女, 彼女がきみにぞっこんで骨, 彼女がぞんざいに投げてよこした眼鏡を実継は慌てて胸元で受け止める, 彼女がデザートを堪能しきった頃ようやく実継は切り出した, 彼女が意識を失ったのはインプラントが侵入前に麻酔措置を施すタイプのものでありそして, 彼女が手を離した, 彼女が持ち歩いていたのだ, 彼女が攻撃衝動抑制インプラントを入れられそうになった時ど, 彼女が気に入り現実にも応用, 彼女が目を細め薄ら笑いを浮かべたからだ, 彼女が遠ざかる足音, 美亜羽へ贈る拳銃
Topic 2: bone, bonephone, phone, そんな風に俺たちの物語を終えられたらどれだけ簡単だっただろう, 兄さん直に会ってソフトで変換していない和弥の声を耳にするのは数年ぶりだった, 兄貴という言葉を拾って耳元の, 兄貴なら, 全身の力が抜けるのが分かった, 分かって, 呆れるほどに四六時中実継とのおしゃべりを望んだ, 喉と舌を潤したくてたまらないがグ, 墓地に火薬の炸裂するような音が響いたからだ, 壊れろ, 変化が止められると思い込んでたらしい, 多くのwkには固有の仕様書が添付さ, 大丈夫,

In [19]:
# Save results to a CSV file
output_file = "miyaha-wd_bert_topic_modeling_results.csv"
topic_sentences.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file}")

Results saved to miyaha-wd_bert_topic_modeling_results.csv


In [25]:
# Download the output file if running in Colab
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>